In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import pandas as pd
from energySystem import system, NaturalGasFurnace
import default_values as DEFAULT

## Import data

In [24]:
def get_demand_mask_and_price(df, colName, dt):
    col = df[colName].to_numpy() / dt
    mask = col > 0
    price = col[mask][0]
    return mask, price

In [25]:
##### loads
df = pd.read_csv("site1_load.csv")
dt = 1/4 # in hours
powerLoad, heatLoad = df['Power Load [kWh]'].to_numpy(), df['Heat Load [kWh]'].to_numpy()

##### prices
df = pd.read_csv("power_price_B20.csv")
energyPricePower = df["energyPrice"].to_numpy() # $/kWh
# from per kW to per kWh in 15 minutes
powerDemand = [get_demand_mask_and_price(df, colName, dt) for colName in ["peakDemandSummerPrice",
                                                                          "partialPeakDemandSummerPrice",
                                                                          "demandSummerPrice",
                                                                          "peakDemandWinterPrice",
                                                                          "demandWinterPrice"]]
df = pd.read_csv("gas_price.csv")
energyPriceGas = df["energyPrice"].to_numpy() # $/kWh

##### emissions
df = pd.read_csv("power_grid_emissions.csv")
df.ffill(inplace=True)
emissionsPower = df["MOER version 2.0"].to_numpy() # kCO2eq/KWhe
df = pd.read_csv("gas_emissions.csv")
emissionsGas = df["gasEmissions"].to_numpy() # kCO2eq/kWhgas

##### time index
timeIndex = pd.date_range(start='1/1/2023', periods=len(powerLoad), freq='15min')

# clear memory
del df

In [26]:
# number of timesteps
n_timesteps = len(powerLoad)

## Natural Gas Only

In [44]:
system1 = system(
    "baseline",
    timeIndex=timeIndex,
    powerLoad=powerLoad,
    heatLoad=heatLoad,
    powerPrice=energyPricePower,
    gasPrice=energyPriceGas,
    powerMarginalEmissions=emissionsPower,
    gasMarginalEmissions=emissionsGas
)
ngf = NaturalGasFurnace(
    n_timesteps=n_timesteps,
    dt=1/4,
    capacityPrice=DEFAULT.FURNACE_CAPA_PRICE,
    eff=DEFAULT.FURNACE_EFF,
    discRate=DEFAULT.DISCOUNT_RATE,
    n_years=DEFAULT.FURNACE_LIFETIME)
system1.addComponent(ngf)
system1.solve(objective="cost", solver="MOSEK")
system1.describe(detailed=True)


System: baseline
1 component(s)
Component: NaturalGasFurnace
    capacityPrice: 1000
    eff: 0.85
Status: optimal

Annual power consumption: 10000.0 MWh
Annual gas consumption: 34521.0 kWh
Annual cost: 3.862 M$
Annual emissions: 11.65 MtonCO2
LCOE (Electricity): 0.225 $/kWh
LCOH (Heat): 0.055 $/kWh
Carbon Intensity of Electricity: 0.367 kgCO2/kWhe
Carbon Intensity of Heat: 0.272 kgCO2/kWhth

Runs from 2023-01-01 00:00:00 to 2023-12-31 23:45:00
Annual power load: 10000.0 MWh
Annual heat load: 29343.0 MWh
Average supply power price: 0.153 $/kWh
Average supply gas price: 0.046 $/kWh
Average supply power emissions: 0.366 kgCO2/kWhe
Average supply gas emissions: 0.231 kgCO2/kWhth
Annualized capex: 0.701 M$
Power opex: 1.545 M$
Gas opex: 1.615 M$
Total power emissions: 3.67 MtonCO2
Total gas emissions: 7.97 MtonCO2
